### Grupo 5

* Vinícius Matos 
* Vinícius Carneiro 
* João Gabriel Kruschewsky
* Pedro Barbosa
* Éricles Araújo
* Kleber Medeiros


###Criação do dataset IMDB

Utilizando as técnicas de raspagem de dados apresentadas na aula, crie um dataset com as informações dos filmes de longa metragem lançados no primeiro semestre de 2021. Os dados estão disponíveis no site do IMDB e podem ser obtidos a partir da busca avançada do site. Lembrem-se de marcar "Feature film" no campo "Title Type" da página de configuração da busca.

Os dados requeridos são: Título, ano de lançamento, censura, duração, categorias, avaliação, metascore, resumo, número de votos e arrecadação.

Vocês devem entregar aqui o código da raspagem e o dataset construído em csv. 


A atividade deve pode ser feita em equipes. Basta um membro da equipe postar e informar os demais membros no notebook.


In [ ]:
"""
class = lister-item-header 
    Título,  
    ano de lançamento, class = lister-item-header > span > item-year

lister-item-content > text-muted
    censura, certificate
    duração, runtime
    categorias, genre


class = 'rating rating -list'

    avaliação, ratingValue
    número de votos, ratingCounting

class = 'ratings-metascore'
    metascore, metascore favorable

p class = 'text-muted':
    resumo, 'text-muted '

class = 'sort-num_votes-visible'
    arrecadação, 'nv' > 'data-value'
"""



#ver o número de filmes, 

#partir ele em grupos de 250

#fazer requests precisa estar dentro de um try catch block com while

# para cada grupo, fazer o request 

    # para cada filme no grupo, 
        #Verificar se a variável existe (ver se a informação volta, try catch block)
        #Colocar a info num data frame


#exportar em um csv

"\nclass = lister-item-header \n    Título,  \n    ano de lançamento, class = lister-item-header > span > item-year\n\nlister-item-content > text-muted\n    censura, certificate\n    duração, runtime\n    categorias, genre\n\n\nclass = 'rating rating -list'\n\n    avaliação, ratingValue\n    número de votos, ratingCounting\n\nclass = 'ratings-metascore'\n    metascore, metascore favorable\n\np class = 'text-muted':\n    resumo, 'text-muted '\n\nclass = 'sort-num_votes-visible'\n    arrecadação, 'nv' > 'data-value'\n"

In [ ]:
from bs4 import BeautifulSoup
from google.colab import drive
import requests
import pandas as pd
import numpy as np
import re

In [ ]:
from types import FunctionType
def existe(session):
    ### Verifica se a session foi passada ou não, e cria uma se necessário. ###
    headers = {"Accept-Language": "en-US,en;q=0.5"}
    if session == None: 
        session = requests.session()
        session.headers.update(headers)
    return session

def getRequest(page_start=1,NUMBER_OF_FILMS_PER_PAGE=250,session=None,counter = 0):
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2021-01-01,2021-06-30&count="+str(NUMBER_OF_FILMS_PER_PAGE)+"&start="+str(page_start)+"&ref_=adv_nxt"
    session = existe(session)
    response = session.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'html.parser')
    elif counter < 50 :
        print(response)
        counter += 1 
        return getRequest(session,counter)
    else:
        raise Exception("Sorry, After 50 tries, no data was fetched. Final Status: " + str(response.status_code)) 

def return_on_failure():
  def decorate(method):
    def applicator(*args, **kwargs):
      try:
         return method(*args,**kwargs)
      except:
         return np.NaN

    return applicator
  return decorate

@return_on_failure()
def getTotalFilmNumber():
    total_film_number = getRequest().find('div', class_='desc').find('span').get_text().split(" ")[2]
    return int(total_film_number.replace(",",""))

@return_on_failure()
def getTitle (film_number, film):
    title = film.find('h3', class_='lister-item-header').find('a', href=True).get_text() 
    return title

@return_on_failure()
def getYear (film_number, film):
    year = film.find('h3', class_='lister-item-header').find('span', class_='lister-item-year').get_text()
    return year

@return_on_failure()
def getRatingValue (film_number, film):
    ratingValue = film.find('div', class_='ratings-bar').find('div', class_='rating rating-list').find('meta')['content'], 
    return ratingValue

@return_on_failure()
def getRatingCount (film_number, film):
    ratingCount = film.find('div', class_='ratings-bar').find('div', class_='rating rating-list').find_all('meta')[2]['content']
    return ratingCount

@return_on_failure()
def getMetascore (film_number, film):
    metascore = film.find('div', class_='ratings-bar').find('div', class_='ratings-metascore').find('span').get_text().strip()
    return metascore

@return_on_failure()
def getRestriction (film_number, film):
    restriction = film.find('p', class_='text-muted').find('span', class_='certificate').get_text()
    return restriction

@return_on_failure()
def getDuration (film_number, film):
    duration = film.find('p', class_='text-muted').find(lambda tag:tag.name=="span" and "min" in tag.text).get_text()
    return duration

@return_on_failure()
def getGenre (film_number, film):
    genre = film.find('p', class_='text-muted').find('span', class_='genre').get_text().strip()
    return genre

@return_on_failure()
def getDescription (film_number, film):
    description = film.find_all('p', class_='text-muted')[1].get_text().strip()
    return description

@return_on_failure()
def getGross (film_number, film):
    gross = film.find('p', class_='sort-num_votes-visible').find_all("span", {"name":"nv"})[1]["data-value"]
    return gross

def getEmptyDicFilms():
    return     {
                'Titulo':     [], 
                'Ano':        [], 
                'Censura':    [], 
                'Duracao':    [], 
                'Categorias': [],
                'Avaliacao':  [],
                'Metascore':  [],
                'Resumo':     [],
                'Votos':      [],
                'Arrecadacao':[],
                }

def add_film (film_number, page, dict_films):
    film = page.find_all('div', class_='lister-item-content')[film_number]
    dict_films['Titulo'].append(getTitle          (film_number, film))
    dict_films['Ano'].append(getYear              (film_number, film))
    dict_films['Censura'].append(getRestriction   (film_number, film))
    dict_films['Duracao'].append(getDuration      (film_number, film))
    dict_films['Categorias'].append(getGenre      (film_number, film))
    dict_films['Avaliacao'].append(getRatingValue (film_number, film))
    dict_films['Metascore'].append(getMetascore   (film_number, film))
    dict_films['Resumo'].append(getDescription    (film_number, film))
    dict_films['Votos'].append(getRatingCount     (film_number, film))
    dict_films['Arrecadacao'].append(getGross     (film_number, film)) 
    return dict_films

def getNumberOfFilmsPerPage (is_last_group, MAX_NUMBER_OF_FILMS_PER_PAGE, last_page_film_number):
    number_of_films_in_page = MAX_NUMBER_OF_FILMS_PER_PAGE
    if is_last_group:
        number_of_films_in_page = last_page_film_number
    return number_of_films_in_page

def addAllFilmsInPage (page, dict_films, number_of_films_in_page):
    for film_number in range(number_of_films_in_page):
        print(f'film number: {film_number} out of {number_of_films_in_page - 1}')
        dict_films = add_film(film_number, page, dict_films)
    return dict_films

def getAllFilms (number_of_pages, MAX_NUMBER_OF_FILMS_PER_PAGE, last_page_film_number):
    page_start = 1
    dict_films = getEmptyDicFilms()    
    for page_number in range(number_of_pages):
        print(f'page {page_number} out of {number_of_pages - 1}')
        page = getRequest(page_start)
        number_of_films_in_page = getNumberOfFilmsPerPage ((page_number == (number_of_pages - 1)), MAX_NUMBER_OF_FILMS_PER_PAGE, last_page_film_number)
        dict_films = addAllFilmsInPage (page, dict_films, number_of_films_in_page)
        page_start += number_of_films_in_page
    return dict_films

def getFilmsDict ( MAX_NUMBER_OF_FILMS_PER_PAGE = 250 ):
    total_film_number = getTotalFilmNumber()-1
    number_of_pages = (total_film_number // MAX_NUMBER_OF_FILMS_PER_PAGE) + 1
    last_page_film_number = (total_film_number % MAX_NUMBER_OF_FILMS_PER_PAGE)
    return getAllFilms (number_of_pages, MAX_NUMBER_OF_FILMS_PER_PAGE, last_page_film_number)

def getCorrectYearFormat (df):
    df['Ano'] = [(re.findall('\d{4}', years)) for years in df['Ano']]
    df['Ano'] = [str(years).replace("[",'').replace("]",'') for years in df['Ano']]

def getCorrectRatingValueFormat (df):
    df['Avaliacao'] = [str(rating).replace('(','').replace(')','').replace('(','').replace(',','').replace("'",'')
                      for rating in df['Avaliacao']]

def getCSV (df):
    drive.mount('drive', force_remount=True)
    df.to_csv('drive/My Drive/test.csv')
    return None

def treatingDatasetValues (df):
    getCorrectRatingValueFormat(df)
    getCorrectYearFormat(df)
    df = pd.DataFrame(df)
    return getCSV(df)

# Running code
final_dict = getFilmsDict()
treatingDatasetValues(final_dict)


page 0 out of 18
film number: 0 out of 249
film number: 1 out of 249
film number: 2 out of 249
film number: 3 out of 249
film number: 4 out of 249
film number: 5 out of 249
film number: 6 out of 249
film number: 7 out of 249
film number: 8 out of 249
film number: 9 out of 249
film number: 10 out of 249
film number: 11 out of 249
film number: 12 out of 249
film number: 13 out of 249
film number: 14 out of 249
film number: 15 out of 249
film number: 16 out of 249
film number: 17 out of 249
film number: 18 out of 249
film number: 19 out of 249
film number: 20 out of 249
film number: 21 out of 249
film number: 22 out of 249
film number: 23 out of 249
film number: 24 out of 249
film number: 25 out of 249
film number: 26 out of 249
film number: 27 out of 249
film number: 28 out of 249
film number: 29 out of 249
film number: 30 out of 249
film number: 31 out of 249
film number: 32 out of 249
film number: 33 out of 249
film number: 34 out of 249
film number: 35 out of 249
film number: 36 out o